In [1]:
import numpy as np
from typing import List, Tuple

def interval_vikor(
    matrix: np.ndarray,          # Матрица решений: [альтернативы, критерии, 2] (нижняя и верхняя граница)
    weights: List[float],        # Веса критериев (сумма = 1)
    types: List[int],            # Направление критериев: 1 (выгода), -1 (затрат)
    v: float = 0.5               # Коэффициент стратегии (v > 0.5 - большинство; v < 0.5 - меньшинство)
) -> dict:
    """
    Реализация метода Interval VIKOR.
    
    Returns:
        Словарь с ключами:
        - 'ranking': Ранжирование альтернатив по возрастанию Q (индекс 0 - лучшая)
        - 'Q': Интервальные значения Q_i для каждой альтернативы [Q_L, Q_R]
        - 'S': Интервальные значения S_i
        - 'R': Интервальные значения R_i
        - 'best_alternative': Индекс лучшей альтернативы (или список при условии приемлемости)
    """
    
    # ===== 1. ПОДГОТОВКА ДАННЫХ =====
    m, n = matrix.shape[0], matrix.shape[1]  # m альтернатив, n критериев
    
    # Разделяем матрицу на нижние и верхние границы интервалов
    L = matrix[:, :, 0]  # lower bounds
    U = matrix[:, :, 1]  # upper bounds
    
    # ===== 2. ОПРЕДЕЛЕНИЕ ИДЕАЛЬНОЙ (f*) И НАДИРНОЙ (f-) ТОЧЕК =====
    # Для каждого критерия j находим лучшие и худшие значения среди всех альтернатив
    f_star = np.zeros((n, 2))   # Идеальная точка [нижняя, верхняя]
    f_minus = np.zeros((n, 2))  # Надирная точка [нижняя, верхняя]
    
    for j in range(n):
        if types[j] == 1:  # Критерий выгоды (максимизация)
            f_star[j, 0] = np.max(L[:, j])   # Лучшая нижняя граница
            f_star[j, 1] = np.max(U[:, j])   # Лучшая верхняя граница
            f_minus[j, 0] = np.min(L[:, j])  # Худшая нижняя граница
            f_minus[j, 1] = np.min(U[:, j])  # Худшая верхняя граница
        else:  # Критерий затрат (минимизация)
            f_star[j, 0] = np.min(L[:, j])   # Лучшая нижняя граница
            f_star[j, 1] = np.min(U[:, j])
            f_minus[j, 0] = np.max(L[:, j])  # Худшая нижняя граница
            f_minus[j, 1] = np.max(U[:, j])
    
    # ===== 3. РАСЧЕТ ИНТЕРВАЛЬНЫХ ЗНАЧЕНИЙ S_i и R_i =====
    # S_i = Σ w_j * (d(f_j*, f_ij) / d(f_j*, f_j-))
    # R_i = max [w_j * (d(f_j*, f_ij) / d(f_j*, f_j-))]
    
    # Функция для расстояния между интервалами (евклидово расстояние между центрами)
    def interval_distance(a: np.ndarray, b: np.ndarray) -> float:
        # a и b - массивы формы [2] (нижняя, верхняя граница)
        center_a = (a[0] + a[1]) / 2
        center_b = (b[0] + b[1]) / 2
        return abs(center_b - center_a)
    
    S = np.zeros((m, 2))  # Интервальные S_i [S_L, S_R]
    R = np.zeros((m, 2))  # Интервальные R_i [R_L, R_R]
    
    for i in range(m):  # Для каждой альтернативы
        s_lower, s_upper = 0.0, 0.0
        r_lower_list, r_upper_list = [], []
        
        for j in range(n):  # Для каждого критерия
            # Текущий интервал альтернативы i по критерию j
            f_ij = np.array([L[i, j], U[i, j]])
            
            # Расстояния
            d_star_ij = interval_distance(f_star[j], f_ij)
            d_star_minus = interval_distance(f_star[j], f_minus[j])
            
            # Нормализованное расстояние (избегаем деления на 0)
            if d_star_minus == 0:
                norm_dist = 0
            else:
                norm_dist = d_star_ij / d_star_minus
            
            # Взвешенное значение для критерия j
            weighted_val = weights[j] * norm_dist
            
            # Для S_i - суммируем
            s_lower += weighted_val
            s_upper += weighted_val  # В упрощённом случае границы одинаковы
            
            # Для R_i - собираем в список, потом возьмём max
            r_lower_list.append(weighted_val)
            r_upper_list.append(weighted_val)
        
        S[i, 0] = s_lower
        S[i, 1] = s_upper
        R[i, 0] = np.max(r_lower_list) if r_lower_list else 0
        R[i, 1] = np.max(r_upper_list) if r_upper_list else 0
    
    # ===== 4. НАХОЖДЕНИЕ ИДЕАЛЬНЫХ И НАДИРНЫХ S*, S-, R*, R- =====
    # Для интервалов часто используют подход на основе центров
    S_centers = (S[:, 0] + S[:, 1]) / 2
    R_centers = (R[:, 0] + R[:, 1]) / 2
    
    S_star = np.min(S_centers)
    S_minus = np.max(S_centers)
    R_star = np.min(R_centers)
    R_minus = np.max(R_centers)
    
    # ===== 5. РАСЧЕТ ИНТЕРВАЛЬНЫХ ЗНАЧЕНИЙ Q_i =====
    # Q_i = v * ((S_i - S*) / (S- - S*)) + (1 - v) * ((R_i - R*) / (R- - R*))
    Q = np.zeros((m, 2))  # [Q_L, Q_R]
    
    # Избегаем деления на 0
    S_diff = S_minus - S_star
    R_diff = R_minus - R_star
    
    if S_diff == 0:
        S_diff = 1e-10
    if R_diff == 0:
        R_diff = 1e-10
    
    for i in range(m):
        # Рассчитываем для левой и правой границ отдельно
        # Для левых границ используем левые границы S и R
        q_left = v * ((S[i, 0] - S_star) / S_diff) + (1 - v) * ((R[i, 0] - R_star) / R_diff)
        
        # Для правых границ используем правые границы S и R
        q_right = v * ((S[i, 1] - S_star) / S_diff) + (1 - v) * ((R[i, 1] - R_star) / R_diff)
        
        Q[i, 0] = q_left
        Q[i, 1] = q_right
    
    # ===== 6. РАНЖИРОВАНИЕ И ВЫБОР ЛУЧШЕЙ АЛЬТЕРНАТИВЫ =====
    # Для ранжирования интервалов Q_i используем метод на основе центров
    Q_centers = (Q[:, 0] + Q[:, 1]) / 2
    
    # Ранжирование по возрастанию Q (чем меньше Q, тем лучше)
    ranking = np.argsort(Q_centers)
    
    # ===== 7. ПРОВЕРКА УСЛОВИЙ ПРИЕМЛЕМОСТИ (опционально) =====
    # Условие 1: Приемлемое преимущество
    sorted_centers = Q_centers[ranking]
    DQ = 1.0 / (m - 1) if m > 1 else 1.0
    
    condition1 = (len(sorted_centers) > 1 and 
                  (sorted_centers[1] - sorted_centers[0]) >= DQ)
    
    # Условие 2: Приемлемая стабильность
    # Лучшая по Q должна быть лучшей хотя бы по S или R
    best_idx = ranking[0]
    S_centers_sorted_idx = np.argsort(S_centers)
    R_centers_sorted_idx = np.argsort(R_centers)
    
    condition2 = (best_idx == S_centers_sorted_idx[0] or 
                  best_idx == R_centers_sorted_idx[0])
    
    best_alternative = best_idx
    if not (condition1 and condition2) and m > 1:
        # Если условия не выполняются, возвращаем набор компромиссных решений
        # (в данном случае - две лучшие альтернативы)
        best_alternative = [int(ranking[0]), int(ranking[1])]
    
    # ===== 8. ФОРМИРОВАНИЕ РЕЗУЛЬТАТА =====
    return {
        'ranking': ranking.tolist(),
        'Q': Q.tolist(),
        'S': S.tolist(),
        'R': R.tolist(),
        'best_alternative': best_alternative,
        'conditions_met': condition1 and condition2,
        'Q_centers': Q_centers.tolist(),
        'f_star': f_star.tolist(),
        'f_minus': f_minus.tolist()
    }

# ===== ПРИМЕР ИСПОЛЬЗОВАНИЯ =====
if __name__ == "__main__":
    # Пример: выбор из 4 альтернатив по 3 критериям
    # Каждая ячейка - интервал [нижняя_граница, верхняя_граница]
    # Формат: matrix[альтернатива, критерий, граница]
    
    matrix = np.array([
        # Альтернатива A1
        [[0.7, 0.9], [0.6, 0.8], [0.8, 1.0]],
        # Альтернатива A2  
        [[0.5, 0.7], [0.8, 1.0], [0.6, 0.8]],
        # Альтернатива A3
        [[0.8, 1.0], [0.5, 0.7], [0.7, 0.9]],
        # Альтернатива A4
        [[0.6, 0.8], [0.7, 0.9], [0.5, 0.7]]
    ])
    
    # Веса критериев (сумма = 1)
    weights = [0.3, 0.4, 0.3]
    
    # Направление критериев: 1 = выгода, -1 = затрат
    types = [1, 1, -1]
    
    # Коэффициент стратегии
    v = 0.5  # Компромиссное решение
    
    # Запуск метода Interval VIKOR
    result = interval_vikor(matrix, weights, types, v)
    
    # ===== ВЫВОД РЕЗУЛЬТАТОВ =====
    print("=== РЕЗУЛЬТАТЫ INTERVAL VIKOR ===\n")
    
    print("Идеальная точка (f*):")
    for j, (lower, upper) in enumerate(result['f_star']):
        print(f"  Критерий {j+1}: [{lower:.3f}, {upper:.3f}]")
    
    print("\nНадирная точка (f-):")
    for j, (lower, upper) in enumerate(result['f_minus']):
        print(f"  Критерий {j+1}: [{lower:.3f}, {upper:.3f}]")
    
    print("\nМетрики для альтернатив:")
    print("Alt |     S-интервал     |     R-интервал     |     Q-интервал     | Q_center")
    print("-" * 85)
    
    for i in range(len(matrix)):
        s_lower, s_upper = result['S'][i]
        r_lower, r_upper = result['R'][i]
        q_lower, q_upper = result['Q'][i]
        q_center = result['Q_centers'][i]
        
        print(f"A{i+1}  | [{s_lower:.4f}, {s_upper:.4f}] | [{r_lower:.4f}, {r_upper:.4f}] | "
              f"[{q_lower:.4f}, {q_upper:.4f}] | {q_center:.4f}")
    
    print(f"\nРанжирование (лучшая -> худшая): A{result['ranking'][0]+1}", end="")
    for idx in result['ranking'][1:]:
        print(f" → A{idx+1}", end="")
    
    if isinstance(result['best_alternative'], list):
        print(f"\nЛучшие альтернативы (компромиссный набор): A{result['best_alternative'][0]+1} и A{result['best_alternative'][1]+1}")
        print("Условия приемлемости не выполнены полностью.")
    else:
        print(f"\nЛучшая альтернатива: A{result['best_alternative']+1}")
        print(f"Условия приемлемости: {'Выполнены' if result['conditions_met'] else 'Не выполнены'}")
    
    # Дополнительно: анализ чувствительности к коэффициенту v
    print("\n\n=== АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ К КОЭФФИЦИЕНТУ v ===")
    print("v    | Лучшая альтернатива | Q_центры (A1, A2, A3, A4)")
    print("-" * 60)
    
    for v_test in [0.0, 0.3, 0.5, 0.7, 1.0]:
        test_result = interval_vikor(matrix, weights, types, v_test)
        best_idx = test_result['best_alternative']
        if isinstance(best_idx, list):
            best_str = f"A{best_idx[0]+1}+A{best_idx[1]+1}"
        else:
            best_str = f"A{best_idx+1}"
        
        q_centers = ", ".join([f"{q:.3f}" for q in test_result['Q_centers']])
        print(f"{v_test:.1f}  | {best_str:^20} | [{q_centers}]")

=== РЕЗУЛЬТАТЫ INTERVAL VIKOR ===

Идеальная точка (f*):
  Критерий 1: [0.800, 1.000]
  Критерий 2: [0.800, 1.000]
  Критерий 3: [0.500, 0.700]

Надирная точка (f-):
  Критерий 1: [0.500, 0.700]
  Критерий 2: [0.500, 0.700]
  Критерий 3: [0.800, 1.000]

Метрики для альтернатив:
Alt |     S-интервал     |     R-интервал     |     Q-интервал     | Q_center
-------------------------------------------------------------------------------------
A1  | [0.6667, 0.6667] | [0.3000, 0.3000] | [0.7500, 0.7500] | 0.7500
A2  | [0.4000, 0.4000] | [0.3000, 0.3000] | [0.3500, 0.3500] | 0.3500
A3  | [0.6000, 0.6000] | [0.4000, 0.4000] | [0.9000, 0.9000] | 0.9000
A4  | [0.3333, 0.3333] | [0.2000, 0.2000] | [0.0000, 0.0000] | 0.0000

Ранжирование (лучшая -> худшая): A4 → A2 → A1 → A3
Лучшая альтернатива: A4
Условия приемлемости: Выполнены


=== АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ К КОЭФФИЦИЕНТУ v ===
v    | Лучшая альтернатива | Q_центры (A1, A2, A3, A4)
------------------------------------------------------------
0.